In [173]:
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
import glob
import numpy as np
from sklearn import svm
from skimage import feature
from sklearn.metrics import classification_report,accuracy_score

In [74]:
# import training data
live_train = glob.glob('data/train_live/*.png') #positive
spoof_train = glob.glob('data/train_spoof/*.png') # negative

# import testing data
live_test = glob.glob('data/test_live/*.png') # positive
spoof_test = glob.glob('data/train_spoof/*.png') # negative

In [175]:
live_train_images = []
spoof_train_images = []
live_test_images = []
spoof_test_images = []

# read images from given path and append to the variavles
for path in live_train: live_train_images.append(imread(path))
for path in spoof_train: spoof_train_images.append(imread(path))
for path in live_test: live_train_images.append(imread(path))
for path in spoof_test: spoof_train_images.append(imread(path))

In [176]:
# convert images to np arrays
live_train_images, spoof_train_images = np.asarray(live_train_images), np.asarray(spoof_train_images)
live_test_images, spoof_test_images = np.asarray(live_test_images), np.asarray(spoof_test_images)

In [180]:
# calculate and store hog features of training data
# live_train_hog_features, spoof_train_hog_features = [], []

# for live_img, spoof_img in zip(live_train_images, spoof_train_images):
#     live_train_hog_features.append(hog(live_img, feature_vector = True))
#     spoof_train_hog_features.append(hog(spoof_img, feature_vector = True))
    
    
# calculate and store lbp features of training data
live_train_hog_features, spoof_train_hog_features = [], []

from skimage.color import rgb2gray

# parameters copying from scikit docs
METHOD = 'uniform'
R = 3
P = 8 * R

for live_img, spoof_img in zip(live_train_images, spoof_train_images):
    live_img = rgb2gray(live_img)
    spoof_img = rgb2gray(spoof_img)
    live_train_hog_features.append(local_binary_pattern(live_img, P, R, METHOD))
    spoof_train_hog_features.append(local_binary_pattern(spoof_img, P, R, METHOD))

In [182]:
# calculate and store hog features of test data
live_test_hog_features, spoof_test_hog_features = [], []

# for live_img, spoof_img in zip(live_test_images, spoof_test_images):
#     live_test_hog_features.append(hog(live_img, feature_vector = True))
#     spoof_test_hog_features.append(hog(spoof_img, feature_vector = True))

for live_img, spoof_img in zip(live_test_images, spoof_test_images):
    live_img = rgb2gray(live_img)
    spoof_img = rgb2gray(spoof_img)
    live_test_hog_features.append(local_binary_pattern(live_img, P, R, METHOD))
    spoof_test_hog_features.append(local_binary_pattern(spoof_img, P, R, METHOD))

In [183]:
# define labels 1 for live and 0 for spoof
# i.e. create a list of 200 1's and 200 0's 
labels_train = [[1]]*len(live_train_hog_features)+[[0]]*len(spoof_train_hog_features)

# merge both positive and negative training data
full_train = live_train_hog_features + spoof_train_hog_features

# merge both positive and negative training data
full_test = live_test_hog_features + spoof_test_hog_features

# assign labels for all test data, 1 for positive, 0 for negative
labels_test = [[1]]*len(live_test_hog_features)+[[0]]*len(spoof_test_hog_features)

In [184]:
clf = svm.SVC()
hog_features_train = np.array(full_train)
hog_features_test = np.array(full_test)

# stack features and labels together
data_frame_train = np.hstack((hog_features,labels))

# shuffle train data
np.random.shuffle(data_frame_train)

In [185]:
# split features and label
x_train = data_frame[:len(data_frame),:-1]
y_train = data_frame[:len(data_frame),-1]
x_test = hog_features_test
y_test = labels_test

In [189]:
print(x_train.shape)
print(x_test.shape)

(400, 131868)
(0,)


In [186]:
clf.fit(x_train,y_train)

SVC()

In [187]:
y_pred = clf.predict(x_test)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
y_pred

In [ ]:
accuracy_score(y_test, y_pred)